<a href="https://colab.research.google.com/github/yuukun123/Python/blob/master/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

# Tùy chỉnh để hiển thị biểu đồ đẹp hơn
sns.set(style="whitegrid")

Mounted at /content/drive


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/Final_Cleaned_Database.xlsx")
print(df.to_string())

In [5]:
# Đường dẫn file
file_path = "/content/drive/MyDrive/dataset_osfstorage-archive/NormingData/DelawarePainDatabase_SecondaryDatabase_FaceGenCharacterization.xlsx"
xls = pd.ExcelFile(file_path)

hàm lọc dữ liệu sheet ExpressionAverage


In [47]:
# def clean_expression_sheet(df, required_cols=["Expression"]):
#     # Xóa cột không tên
#     df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

#     # Xóa dòng thiếu cột bắt buộc
#     df = df.dropna(subset=required_cols)

#     # Loại bỏ các biểu cảm không dùng
#     df = df[~df["Expression"].str.lower().str.contains("pms13")]
#     df = df[~df["Expression"].str.lower().str.contains("facegenpro_avgblackface")]

#     # Các cột cần ép kiểu và xử lý NaN
#     numeric_cols = ["ThreatRating", "ConfusionRating", "ThreatComp", "ConfusionComp"]
#     for col in numeric_cols:
#         if col in df.columns:
#             df[col] = df[col].replace([".", ""], np.nan)
#             df[col] = pd.to_numeric(df[col], errors="coerce")
#             df[col] = df[col].fillna(df[col].mean())

#     # Loại bỏ các dòng liên quan face gen pro
#     df = df[~df["Norming Survey"].str.lower().str.contains("facegenpro")]

#     df = df.infer_objects(copy=False)

#     return df


In [49]:
def clean_expression_sheet(df, required_cols=["Expression"]):
    # Xóa cột không tên
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # Xóa dòng thiếu cột bắt buộc
    df = df.dropna(subset=required_cols)

    # Loại bỏ các biểu cảm không dùng
    df = df[~df["Expression"].str.lower().str.contains("pms13")]
    df = df[~df["Expression"].str.lower().str.contains("facegenpro_avgblackface")]

    # ✅ Loại bỏ các dòng liên quan face gen pro TRƯỚC để không ảnh hưởng đến mean
    df = df[~df["Norming Survey"].str.lower().str.contains("facegenpro")]

    # Các cột cần ép kiểu và xử lý NaN
    numeric_cols = ["ThreatRating", "ConfusionRating", "ThreatComp", "ConfusionComp"]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].replace([".", ""], np.nan)
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df[col] = df[col].fillna(df[col].mean())

    df = df.infer_objects(copy=False)
    return df


ExpressionAverages

In [ ]:
df_expr = pd.read_excel(xls, sheet_name="ExpressionAverages")
# print(df_expr.dtypes)
df_expr = clean_expression_sheet(df_expr)

print(df_expr.to_string())

hàm lọc dữ liệu sheet slider_face_gen

In [21]:
def clean_slider_facegen_sheet(df):
    # Xóa các cột không tên (thường là chú thích hoặc rỗng)
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # Loại bỏ cột 'PMS13' nếu có (theo khuyến nghị)
    if "PMS13" in df.columns:
        df = df.drop(columns=["PMS13"])

    # Duyệt qua tất cả các cột trừ 'Slider'
    numeric_cols = df.columns.drop("Slider")
    for col in numeric_cols:
        # Thay "." và chuỗi rỗng thành NaN
        df[col] = df[col].replace([".", ""], np.nan)

        # Ép kiểu thành số (sai sẽ thành NaN)
        df[col] = pd.to_numeric(df[col], errors="coerce")

        # Điền thiếu bằng trung bình cột
        df[col] = df[col].fillna(df[col].mean())

    # Bỏ cảnh báo downcasting
    df = df.infer_objects(copy=False)

    return df


Slider_face_gen

In [ ]:
df_slider_mo = pd.read_excel(xls, sheet_name="Sliders_FaceGenModeller")
# print(df_slider_mo.dtypes)
df_slider_mo = clean_slider_facegen_sheet(df_slider_mo)
print(df_slider_mo.to_string())

hàm lọc first_pass_norming

In [24]:
def clean_first_pass_norming(df):
    # 1. Loại cột pms13
    df = df.loc[:, ~df.columns.str.contains("pms13", case=False)]

    # 2. Tính missing ratio
    rating_cols = [c for c in df.columns if "emotion" in c.lower()]
    miss_ratios = df[rating_cols].isna().mean()

    # 3. Drop cột missing > 0.4
    keep_cols = miss_ratios[miss_ratios <= 0.4].index.tolist()
    df = df[keep_cols + [col for col in df.columns if col not in rating_cols]]

    # 4. Với các cột rating còn lại:
    #    - Nếu missing < 0.05, drop dòng
    #    - Ngược lại impute bằng mode
    small_missing = miss_ratios[miss_ratios < 0.05].index.tolist()
    large_missing = miss_ratios[(miss_ratios >= 0.05) & (miss_ratios <= 0.4)].index.tolist()

    df = df.dropna(subset=small_missing)  # drop rows
    for col in large_missing:
        mode_val = df[col].mode().iat[0]
        df[col] = df[col].fillna(mode_val).astype(int)

    return df

Fist_pass_norming

In [ ]:
df_first = pd.read_excel(xls, sheet_name="FirstPassNorming")
df_first = clean_first_pass_norming(df_first)
print(df_first.to_string())
# df_first.head()


hàm lọc demographics

In [29]:
def clean_demographics(df):
  cols_to_drop = df.columns[6:20]

  # Drop những cột đó
  df = df.drop(columns=cols_to_drop)

  return df

Demographics

In [ ]:
df_demographics = pd.read_excel(xls, sheet_name="Demographics")
df_demographics = clean_demographics(df_demographics)
print(df_demographics.to_string())
